In [1]:
import pandas as pd
import datetime as dt 

In [2]:
def calculate_metrics(conv_df, metric):
    try:
        start = conv_df['date'].iloc[0]
        end = conv_df[conv_df['end_conversation']==True].iloc[0]['date']
        duration = (end-start).seconds
        
        start_rating = conv_df[(conv_df['trigger']=='rating')&(conv_df['reaction']==False)].iloc[0]['date']
        end_rating = conv_df['date'].iloc[-1]
        duration_rating = (end_rating-start_rating).seconds
        duration+=duration_rating
        
    except:
         duration = (conv_df['date'].iloc[-1] - conv_df['date'].iloc[0]).seconds #rating sorusuna kadar gecen sure olmali
    metric['duration'] = duration
    #print(metric['duration'])
    metric['length'] = conv_df.shape[0]
    metric['initiatedByUser'] = not conv_df["from_is_bot"].iloc[0]
    metric['turnsByUser'] = (~conv_df['from_is_bot']).values.sum()
    free_text = conv_df[(conv_df['type']=='message')& (conv_df['is_command']==False)]['text']
    if free_text.any():
        metric['numWords'] = round(free_text.apply(lambda a: len(a.split())).mean(),4)
    else:
        metric['numWords']=0
    
    moodQ=0
    emotionQ =0
    ratingQ=0
    
    mood_ignored=0
    emo_ignored=0
    rating_ignored=0
    
    rr_mood = None
    rr_emotion = None
    rr_rating = None
    rd_mood = None
    rd_emotion = None
    rd_rating = None
    emo_count=0
    
    mask = (conv_df['trigger'] == 'cloud-scheduler') | (conv_df['trigger'] == 'command-save')
    length = conv_df[mask].shape[0]
    moodQ = length
    for index, row in conv_df[mask].iterrows():
        message_id = row["message_id"]
        mood_mask = (conv_df["message_id"] == message_id) & (conv_df["trigger"]=="mood-callback")
        response = conv_df[mood_mask]
        if len(response):
            response = response.iloc[0]
            mood = str(response["data"])
            if rr_mood:
                rr_mood +=1
            else:
                rr_mood = 1
            if rd_mood:
                rd_mood+=(response["date"]-row["date"]).seconds
            else:
                rd_mood = (response["date"]-row["date"]).seconds
            if "bad" in mood:
                emotionQ+=1
                emo_count+=1
                hook = response["hook"]
                emotion_request = conv_df[conv_df["hook"] == message_id].iloc[0]
                m_id = emotion_request["message_id"]
                emotion_response = conv_df[conv_df["message_id"] == m_id].loc[conv_df["trigger"]=="mood-callback"]
                if len(emotion_response):
                    emotion_response= emotion_response.iloc[0]
                    emotion = emotion_response["data"]
                    if emotion:
                        if rd_emotion:
                            rd_emotion +=(emotion_response["date"]-emotion_request["date"]).seconds
                        else:
                            rd_emotion =(emotion_response["date"]-emotion_request["date"]).seconds
                        if rr_emotion:
                            rr_emotion+=1
                        else:
                            rr_emotion=1
                else:
                    emo_ignored+=1
                    print("No response to emotion question", row["chat_id"], row["message_id"])
                    
        else:
            mood_ignored+=1
            print("No response to mood question", row["chat_id"], row["message_id"])
            #mood sorusuna cevap gelmemis demektir
    
    rating_mask = (conv_df['trigger'] == 'rating') & (conv_df['reaction'] == False)
    rating_length = conv_df[rating_mask].shape[0]
    ratingQ=rating_length
    for index, row in conv_df[rating_mask].iterrows():
        message_id = row["message_id"]
        callback_mask = (conv_df["message_id"] == message_id) & (conv_df["trigger"]=="rating-callback")
        rating_response = conv_df[callback_mask]
        if len(rating_response):
            rating_response= rating_response.iloc[0]
            raing = str(rating_response["data"])
            if rr_rating:
                rr_rating +=1
            else:
                rr_rating = 1
            if rd_rating:
                rd_rating+=(rating_response["date"]-row["date"]).seconds
            else:
                rd_rating = (rating_response["date"]-row["date"]).seconds
        else:
            rating_ignored+=1
            print("No response to rating question", row["chat_id"], row["message_id"])
            #rating sorusuna cevap gelmmemis demektir
            
            
    if rr_mood:
        metric["rr_mood"]=rr_mood/length
    else:
        metric["rr_mood"]=float("NaN")
    if rr_emotion:
        metric["rr_emotion"]=rr_emotion/emo_count
    else:
        metric["rr_emotion"]=float("NaN")
    if rr_rating:
        metric["rr_rating"]=rr_rating/rating_length
    else:
        metric["rr_rating"]=float("NaN")
    
    if rd_emotion:
        metric["rd_emotion"]=str(rd_emotion/emo_count).split(".")[0]
    else:
        metric["rd_emotion"]=float("NaN")
    if rd_mood:
        metric["rd_mood"] = str(rd_mood/length).split(".")[0]
    else:
        metric["rd_mood"] = float("NaN")
    if rd_rating:
        metric["rd_rating"] = str(rd_rating/rating_length).split(".")[0]
    else:
        metric["rd_rating"] = float("NaN")
    
    
    
    #RATING
    # Current : length is total number of replies
    # Alternative: length of all answered and not-answered questions 
    # Question: If the user didnt rate, do we include it into average? No.
    r_mask = (conv_df['trigger'] == 'rating-callback')
    r_length = conv_df[r_mask].shape[0]
    avg_rating =0
    if r_length:
        for index, row in conv_df[r_mask].iterrows():
            avg_rating+=int(row["data"])
        avg_rating = avg_rating/r_length
        metric["rating"]=avg_rating
    else:
        metric["rating"] = None 
    
    metric["RR"]=None
    metric["RD"]=None
    rr_total =0
    if rr_rating:
        rr_total+=rr_rating
    if rr_mood:
        rr_total+=rr_mood
    if rr_emotion:
        rr_total += rr_emotion
    
    rd_total =0
    if rd_rating:
        rd_total+= rd_rating
    if rd_mood:
        rd_total+= rd_mood
    if rd_emotion:
        rd_total+=rd_emotion
    
    try:
        metric["RR"] =round(rr_total/(moodQ+emotionQ+ratingQ), 2)
        metric["RD"] =round(rd_total/(moodQ+emotionQ+ratingQ), 2)
    except:
        print("divide by zero maybe")
    
    desc = {
        "mood_ignored":mood_ignored,
        "moodQ":moodQ,
        "emo_ignored":emo_ignored,
        "emotionQ":emotionQ,
        "rating_ignored":rating_ignored,
        "ratingQ":ratingQ
    }
    
    return metric, desc

In [3]:
desc_stats=list()
def get_metrics(df_all):
    all_metrics = list()
    for chat_id, chat_df in df_all.groupby('chat_id'):
        for conv_id, conv_df in chat_df.groupby('conv_id'):
            metric = dict()
            metric['usergroup']= chat_df['usergroup'].iloc[0]
            metric['chat_id']=chat_id
            metric['conv_id']=conv_id
            metric, desc = calculate_metrics(conv_df, metric)
            all_metrics.append(metric)
            desc_stats.append({"group": chat_df['usergroup'].iloc[0],"desc":desc})
    df_metrics = pd.DataFrame.from_records(all_metrics)
    df_metrics.fillna(value=pd.np.nan, inplace=True)
    return df_metrics

In [4]:
df_users = pd.read_csv('users.csv')
df_users

chat_id     from_id                 start_date usergroup schedule  \
0     95008127    95008127  2020-03-21 13:09:04+01:00   default    09:00   
1    162138505   162138505  2020-03-21 13:44:31+01:00    custom    16:00   
2    905572285   905572285  2020-03-21 20:18:08+01:00   default    09:00   
3    891239356   891239356  2020-03-23 11:02:04+01:00    custom    10:00   
4    982818844   982818844  2020-04-10 21:11:42+02:00   default    09:00   
5   1042605234  1042605234  2020-04-10 22:41:21+02:00    custom    15:00   
6   1132498866  1132498866  2020-04-11 19:53:15+02:00   default    09:00   
7   1268414753  1268414753  2020-04-12 21:22:46+02:00    custom    18:00   
8    177635781   177635781  2020-04-13 17:10:06+02:00   default    09:00   
9    391209184   391209184  2020-04-13 17:19:29+02:00    custom    15:00   
10   131217570   131217570  2020-04-13 18:06:02+02:00   default    09:00   
11   929690817   929690817  2020-04-14 10:19:16+02:00    custom    19:00   
12   105121899   105121899  2020-04-14 10:41:39+02:00   default    09:00   
13   152676961   152676961  2020-04-14 19:10:19+02:00    custom    12:00   
14  1172836796  1172836796  2020-04-15 17:27:48+02:00   default    09:00   
15  1112568683  1112568683  2020-04-15 19:58:36+02:00    custom    13:00   
16  1249323037  1249323037  2020-04-15 20:50:55+02:00   default    09:00   
17   533137038   533137038  2020-04-16 02:18:05+02:00    custom    19:00   
18  1127787098  1127787098  2020-04-16 17:51:45+02:00   default    09:00   
19   409808468   409808468  2020-04-16 23:24:06+02:00    custom    12:00   
20  1134481057  1134481057  2020-04-19 20:13:20+02:00   default    09:00   
21   886555239   886555239  2020-04-23 11:16:57+02:00    custom    19:00   
22  1231677107  1231677107  2020-04-25 13:26:35+02:00   default    09:00   
23   869537211   869537211  2020-04-26 10:23:09+02:00    custom    11:00   
24  1210332933  1210332933  2020-05-03 12:55:00+02:00   default    09:00   
25  1234925460  1234925460  2020-05-03 13:26:34+02:00    custom    14:00   
26   818710298   818710298  2020-05-04 19:29:32+02:00   default    09:00   
27  1253643703  1253643703  2020-05-04 19:33:33+02:00    custom    10:00   
28  1133412729  1133412729  2020-05-14 19:25:19+02:00   default    09:00   
29  1018834281  1018834281  2020-05-14 20:16:09+02:00    custom    14:00   
30  1280374122  1280374122  2020-05-16 10:59:52+02:00   default    09:00   
31   863094980   863094980  2020-05-17 09:54:12+02:00    custom    19:00   
32   995172882   995172882  2020-05-18 22:24:22+02:00   default    09:00   
33   279331946   279331946  2020-05-19 00:00:07+02:00    custom    15:00   

                     end_date  inactive_since  usage_time  
0   2020-05-29 10:22:46+02:00               0          69  
1   2020-05-29 17:56:34+02:00               0          70  
2   2020-05-19 11:56:03+02:00              10          59  
3   2020-04-18 13:01:28+02:00              41          27  
4   2020-05-29 16:00:23+02:00               0          49  
5   2020-04-22 00:12:02+02:00              37          12  
6   2020-05-29 17:09:41+02:00               0          48  
7   2020-05-29 18:33:56+02:00               0          47  
8   2020-05-06 22:00:18+02:00              22          24  
9   2020-05-29 16:07:36+02:00               0          46  
10  2020-04-15 11:34:04+02:00              44           2  
11  2020-04-22 16:42:27+02:00              37           9  
12  2020-04-19 12:32:21+02:00              40           6  
13  2020-05-23 10:16:06+02:00               6          39  
14  2020-05-18 11:56:29+02:00              11          33  
15  2020-05-14 13:11:59+02:00              15          29  
16  2020-04-21 10:27:59+02:00              38           6  
17  2020-05-03 19:22:16+02:00              26          18  
18  2020-04-18 09:45:27+02:00              41           2  
19  2020-04-17 12:15:57+02:00              42           1  
20  2020-04-22 09:00:52+02:00              37           3  
21  2020-05-29 20:13:54+02

In [5]:
df_messages = pd.read_csv('messages.csv')
df_messages['date'] = df_messages['date'].apply(lambda a:dt.datetime.strptime(a[:19], '%Y-%m-%d %H:%M:%S'))
df_messages['date'] = pd.to_datetime(df_messages['date'])
df_messages.head()

chat_id    from_id  conv_id  \
0  95008127   95008127        1   
1  95008127  939263885        1   
2  95008127  939263885        1   
3  95008127   95008127        1   
4  95008127  939263885        1   

                                                text                date  \
0                                             /start 2020-03-21 13:09:04   
1  Welcome to the Mood Tracker! I am here to keep... 2020-03-21 13:09:05   
2                            How is your mood today? 2020-03-21 13:09:06   
3                                                NaN 2020-03-21 13:09:09   
4              I am glad to hear that you feel good! 2020-03-21 13:09:10   

   data  message_id  hook        trigger  reaction  end_conversation  \
0   NaN        2491     0            NaN     False             False   
1   NaN        2492     0  command-start     False             False   
2   NaN        2493     0   command-save     False             False   
3  good        2493     0  mood-callback     False             False   
4   NaN        2494     0  feedback-mood      True              True   

   from_is_bot            type  is_command usergroup  
0        False         message        True   default  
1         True          result       False   default  
2         True          result       False   default  
3        False  callback_query       False   default  
4         True          result       False   default

In [6]:
df_metrics = get_metrics(df_messages)
df_metrics.head()

No response to mood question 95008127 3074
No response to mood question 95008127 3201
No response to rating question 105121899 3853
No response to rating question 105121899 3860
No response to rating question 105121899 4170
No response to rating question 105121899 4296
No response to mood question 105121899 4370
No response to mood question 105121899 4455
No response to rating question 131217570 3442
No response to rating question 131217570 3630
No response to rating question 131217570 3699
No response to rating question 131217570 3711
No response to mood question 131217570 3824
No response to mood question 131217570 4003
No response to mood question 131217570 4154
No response to mood question 131217570 4270
No response to mood question 131217570 4369
No response to mood question 131217570 4454
No response to mood question 131217570 4517
No response to mood question 131217570 4622
No response to mood question 131217570 4739
No response to mood question 131217570 4859
No response to moo

No response to mood question 891239356 2777
No response to mood question 891239356 2814
No response to rating question 891239356 2857
No response to rating question 891239356 2887
No response to rating question 891239356 3028
No response to mood question 891239356 3076
No response to rating question 891239356 3160
No response to mood question 891239356 3161
No response to mood question 891239356 3172
No response to mood question 891239356 3186
No response to mood question 891239356 3270
No response to mood question 891239356 3318
No response to mood question 891239356 3345
No response to mood question 891239356 3530
No response to mood question 891239356 3670
No response to mood question 891239356 3837
No response to mood question 891239356 4044
No response to mood question 891239356 4279
No response to mood question 891239356 4391
No response to mood question 891239356 4474
No response to mood question 891239356 4529
No response to mood question 891239356 4630
No response to mood ques

No response to rating question 1112568683 5595
No response to rating question 1112568683 5694
No response to rating question 1112568683 5745
No response to rating question 1112568683 5793
No response to rating question 1112568683 6039
No response to rating question 1112568683 6132
No response to rating question 1112568683 6196
No response to rating question 1112568683 6249
No response to rating question 1112568683 6525
No response to rating question 1112568683 6573
No response to rating question 1112568683 6651
No response to rating question 1112568683 6725
No response to rating question 1112568683 6833
No response to rating question 1112568683 6919
No response to mood question 1112568683 7031
No response to mood question 1112568683 7151
No response to mood question 1112568683 7245
No response to mood question 1112568683 7344
No response to mood question 1112568683 7472
No response to mood question 1112568683 7600
No response to mood question 1112568683 7712
No response to mood questio

No response to mood question 1280374122 7305
No response to mood question 1280374122 7429
No response to mood question 1280374122 7555
No response to mood question 1280374122 7684
No response to rating question 1280374122 7875
No response to mood question 1280374122 7996
No response to mood question 1280374122 8075
No response to mood question 1280374122 8182
No response to mood question 1280374122 8255
No response to mood question 1280374122 8340
No response to mood question 1280374122 8411


RD   RR   chat_id  conv_id  duration  initiatedByUser  length  \
0    9.50  1.0  95008127        1        23             True       8   
1   11.00  1.0  95008127        2        77             True       8   
2   19.67  1.0  95008127        3        66            False      11   
3  164.00  1.0  95008127        4       844             True       9   
4  152.67  1.0  95008127        5       490             True      11   

   numWords  rating rd_emotion rd_mood rd_rating  rr_emotion  rr_mood  \
0       0.0     4.0        NaN       3        16         NaN      1.0   
1       2.0     5.0        NaN     NaN        11         NaN      NaN   
2       0.0     4.0          8      16        35         1.0      1.0   
3       4.0     5.0        NaN     NaN       164         NaN      NaN   
4       4.0     4.0          5     450         3         1.0      1.0   

   rr_rating  turnsByUser usergroup  
0        1.0            3   default  
1        1.0            3   default  
2        1.0            4   default  
3        1.0            4   default  
4        1.0            5   default

In [7]:
print(len(df_metrics))

762


In [8]:
df_metrics_compact=df_metrics[["usergroup","chat_id","conv_id","RR","RD","rating","duration", "initiatedByUser", "length", "turnsByUser","numWords" ]]
df_metrics_compact # Fillna then save

usergroup     chat_id  conv_id    RR        RD  rating  duration  \
0     default    95008127        1  1.00      9.50     4.0        23   
1     default    95008127        2  1.00     11.00     5.0        77   
2     default    95008127        3  1.00     19.67     4.0        66   
3     default    95008127        4  1.00    164.00     5.0       844   
4     default    95008127        5  1.00    152.67     4.0       490   
5     default    95008127        6  1.00      3.00     4.0        54   
6     default    95008127        7  1.00   1438.50     5.0      2878   
7     default    95008127        8  1.00     17.00     4.0        33   
8     default    95008127        9  1.00      9.00     4.0        11   
9     default    95008127       10  1.00    186.50     5.0       378   
10    default    95008127       11  1.00     11.00     5.0      1834   
11    default    95008127       12  1.00     16.00     4.0        26   
12    default    95008127       13  1.00      3.00     4.0       268   
13    default    95008127       14  1.00     23.67     4.0        86   
14    default    95008127       15  1.00     12.00     5.0        40   
15    default    95008127       16  1.00     12.00     5.0        51   
16    default    95008127       17  1.00     14.50     5.0       577   
17    default    95008127       18  1.00     14.00     5.0        30   
18    default    95008127       19  1.00    113.00     4.0       228   
19    default    95008127       20  1.00     14.00     5.0        30   
20    default    95008127       21  1.00   3446.00     4.0     10340   
21    default    95008127       22  1.00   6372.67     5.0     19121   
22    default    95008127       23  1.00   6538.50     5.0     13079   
23    default    95008127       24  1.00   4475.00     4.0      8952   
24    default    95008127       25  1.00    127.00     4.0       140   
25    default    95008127       26  1.00      9.00     5.0        12   
26    default    95008127       27  1.00    362.00     4.0       425   
27    default    95008127       28  1.00    443.00     5.0      2724   
28    default    95008127       29  1.00      9.00     4.0      1128   
29    default    95008127       30  1.00    497.00     4.0      2846   
..        ...         ...      ...   ...       ...     ...       ...   
732    custom  1268414753       16  1.00    142.00     5.0       285   
733    custom  1268414753       17  1.00    491.67     5.0      1477   
734    custom  1268414753       18  1.00    139.33     5.0       325   
735    custom  1268414753       19  1.00    433.50     5.0       867   
736    custom  1268414753       20  1.00    948.00     5.0      1898   
737    custom  1268414753       21  1.00    370.50     5.0       742   
738    custom  1268414753       22  1.00    348.50     5.0       698   
739    custom  1268414753       23  1.00    529.83     5.0      1129   
740    custom  1268414753       24  1.00    682.75     5.0      2644   
741    custom  1268414753       25  1.00    184.00     5.0       371   
742    custom  1268414753       26  1.00    541.00     5.0      1083   
743    custom  1268414753       27  1.00  21508.75     5.0     85642   
744    custom  1268414753       28  1.00    775.50     5.0      1552   
745    custom  1268414753       29  1.00   1472.50     5.0      2946   
746    custom  1268414753       30  1.00   1663.50     5.0      3327   
747    custom  1268414753       31  1.00    666.50     5.0      1336   
748    custom  1268414753       32  1.00    625.67     5.0      1252   
749    custom  1268414753       33  1.00    151.33     5.0       455   
750    custom  1268414753       34  1.00   1075.50     5.0      2151   
751    custom  1268414753       35  1.00    565.00     5.0      1131   
752    custom  1268414753       36  1.00    723.00     5.0      1447   
753    custom  1268414753       37  1.00  43192.50     5.0     86386   
754    custom  1268414753       38  1.00    659.00     5.0      1318   
755    custom  1268414753       39  1.00  43

In [9]:
mood_total=0
mood_ignored=0
emo_total=0
emo_ignored=0
rating_ignored=0
rating_total=0

custom_mood_total=0
custom_mood_ignored=0
custom_emo_total=0
custom_emo_ignored=0
custom_rating_ignored=0
custom_rating_total=0
for conv in desc_stats:
    group = conv.get("group")
    desc=conv.get('desc')
    if group =='default':
        mood_ignored+=desc.get("mood_ignored")
        mood_total+=desc.get("moodQ")
        emo_ignored+=desc.get("emo_ignored")
        emo_total+=desc.get("emotionQ")
        rating_ignored+=desc.get("rating_ignored")
        rating_total+=desc.get("ratingQ")
    else:
        custom_mood_ignored+=desc.get("mood_ignored")
        custom_mood_total+=desc.get("moodQ")
        custom_emo_ignored+=desc.get("emo_ignored")
        custom_emo_total+=desc.get("emotionQ")
        custom_rating_ignored+=desc.get("rating_ignored")
        custom_rating_total+=desc.get("ratingQ")

ignored = [
    {"M":mood_total, "MI":mood_ignored, "E":emo_total, "EI": emo_ignored, 
     "R":rating_total, "RI":rating_ignored, "usergroup":"default"},
    {"M":custom_mood_total, "MI":custom_mood_ignored, "E":custom_emo_total, "EI": custom_emo_ignored, 
     "R":custom_rating_total, "RI":custom_rating_ignored, "usergroup":"custom"}
]

ignored_df = pd.DataFrame(ignored)
ignored_df

E  EI    M   MI    R  RI usergroup
0  94   1  621  213  367  66   default
1  69   1  665  267  377  57    custom

In [10]:
df_metrics.to_csv('metrics_raw.csv', index=False)

In [11]:
df_metrics_compact.to_csv('metrics_compact.csv', index=False)

In [12]:
ignored_df.to_csv('ignored.csv', index=False)